<h2>Part 1 - Extract and Transform</h2>

In [59]:
import json
import csv
import requests
from normalise import normalise
import pandas as pd

In [29]:
# call api request to get json data
resp = requests.get('https://maps2.bristol.gov.uk/server2/rest/services/ext/air_quality/MapServer/0/query?outFields=*&where=1%3D1&f=geojson')

# convert to response as python
air_quality_data = resp.json()

<h3>1. Retrieve data from the API</h3>

In [30]:
# write to new file
with open("air_quality_data.json","w") as outfile:
    json.dump(air_quality_data, outfile, indent=4)

In [31]:
# write to new file
with open("air_quality_data.json","w") as outfile:
    # loop through the rows
    for row in air_quality_data['features']:
        # create new column in line with rest of columns for coordinates 
        row[u'latitude']=row[u'geometry'][u'coordinates'][0]
        row[u'longitude']=row[u'geometry'][u'coordinates'][1]
        # create new columns for each row in properties dictionary
        for column in row['properties']:
            row[column]=row[u'properties'][column]
        # delete the geometry & properties dictionaries
        del row[u'geometry']
        del row[u'properties']
    # saving data as local json file
    json.dump(air_quality_data, outfile, indent=4)

In [32]:
# create pandas dataframe
features_df = pd.DataFrame(air_quality_data['features'])

# describe the dataframes
features_df.describe()

,id,latitude,longitude,OBJECTID,SiteID,Easting,Northing,InstrumentID,tube_kerb,rec_kerb,Tube_Height,Elevation,Grid_ID,RouteSequenceID,DateStart,DateEnd,Colocated,obstruction_m,junction_m
count,700.00000,700.000000,700.000000,700.00000,700.000000,700.000000,700.000000,700.000000,322.000000,321.000000,332.000000,687.000000,596.000000,93.0,1.670000e+02,1.000000e+01,8.00000,201.000000,201.000000
mean,350.50000,-2.596000,51.463203,350.50000,357.857143,358692.705714,173972.308571,1.187143,3.440062,18.295639,2.371084,25.848908,57.243289,0.0,1.573555e+12,1.301988e+12,409.50000,2.308458,98.970149
std,202.21688,0.043134,0.027309,202.21688,205.861016,2987.707553,3045.318301,0.766319,7.198700,33.199122,0.449916,23.863167,27.861715,0.0,1.573320e+11,1.707026e+11,164.78124,2.185035,91.091378
min,1.00000,-2.841915,51.386765,1.00000,1.000000,341567.000000,165538.000000,1.000000,0.000000,0.000000,1.500000,-3.800000,0.000000,0.0,8.843904e+11,9.497952e+11,203.00000,1.000000,1.000000
25%,175.75000,-2.605547,51.446782,175.75000,178.750000,358014.000000,172138.750000,1.000000,1.000000,3.000000,2.000000,9.500000,41.750000,0.0,1.609718e+12,1.258222e+12,256.25000,1.000000,32.000000
50%,350.50000,-2.589323,51.457948,350.50000,360.500000,359142.500000,173375.000000,1.000000,2.000000,6.000000,2.400000,15.000000,56.000000,0.0,1.609718e+12,1.357258e+12,457.50000,1.000000,75.000000
75%,525.25000,-2.572071,51.475393,525.25000,536.250000,360350.000000,175316.000000,1.000000,3.000000,11.700000,2.500000,36.650000,67.000000,0.0,1.609718e+12,1.431799e+12,500.25000,3.000000,145.000000
max,700.00000,-2.366796,51.617494,700.00000,711.000000,374636.000000,191132.000000,10.000000,100.000000,222.000000,6.000000,228.200000,144.000000,0.0,1.722470e+12,1.451520e+12,672.00000,15.000000,540.000000


In [33]:
# checking 5 values from the dataframe
features_df.head(5)

,type,id,latitude,longitude,OBJECTID,siteNo,location,SiteID,Easting,Northing,...,RouteSequenceID,Description,DateStart,DateEnd,Colocated,Duplicate_Triplicate,obstruction_m,obstruction_lt,inlet_flow_clear,junction_m
0,Feature,1,-2.627749,51.407746,1,1,Withywood School,1,356434,167823,...,NaN,,NaN,NaN,NaN,,NaN,No,No,NaN
1,Feature,2,-2.596814,51.454557,2,B1,Colston Avenue,2,358628,173011,...,NaN,,NaN,NaN,NaN,,2.0,No,Yes,1.0
2,Feature,3,-2.613994,51.469205,3,B10,Blackboy Hill,3,357448,174650,...,NaN,,NaN,NaN,NaN,,1.0,Yes,No,32.0
3,Feature,4,-2.578333,51.444210,4,B11,Three Lamps,4,359903,171850,...,NaN,,NaN,NaN,NaN,,4.0,No,Yes,1.0
4,Feature,5,-2.595294,51.442812,5,B12,Bedminster Parade,5,358723,171704,...,NaN,,NaN,NaN,NaN,,1.0,Yes,No,96.0


<h3>2. Data Transformation</h3>

In [37]:
# finding max/min values for latitude and longitude
max_long = features_df['longitude'].max()
min_long = features_df['longitude'].min()
max_lat = features_df['latitude'].max()
min_lat = features_df['latitude'].min()

print('The minimum value for Longitude is: ', max_long)
print('The maximum value for Longitude is: ', min_long)

print('The minimum value for Latitude is: ', max_lat)
print('The maximum value for Latitude is: ', min_lat) 

The minimum value for Longitude is:  51.61749401500854
The maximum value for Longitude is:  51.38676459040267
The minimum value for Latitude is:  -2.366795987602053
The maximum value for Latitude is:  -2.841914759125154


In [38]:
# normalise column for positive entry
features_df['Latitude (Normalised)'] = features_df.latitude.apply(normalise, min=-90, max=90)

# normalise column for negative entry
features_df['Longitude (Normalised)'] = features_df.longitude.apply(normalise, min=-180, max=180)

features_df

,type,id,latitude,longitude,OBJECTID,siteNo,location,SiteID,Easting,Northing,...,DateStart,DateEnd,Colocated,Duplicate_Triplicate,obstruction_m,obstruction_lt,inlet_flow_clear,junction_m,Latitude (Normalised),Longitude (Normalised)
0,Feature,1,-2.627749,51.407746,1,1,Withywood School,1,356434,167823,...,NaN,NaN,NaN,,NaN,No,No,NaN,0.485401,0.642799
1,Feature,2,-2.596814,51.454557,2,B1,Colston Avenue,2,358628,173011,...,NaN,NaN,NaN,,2.0,No,Yes,1.0,0.485573,0.642929
2,Feature,3,-2.613994,51.469205,3,B10,Blackboy Hill,3,357448,174650,...,NaN,NaN,NaN,,1.0,Yes,No,32.0,0.485478,0.642970
3,Feature,4,-2.578333,51.444210,4,B11,Three Lamps,4,359903,171850,...,NaN,NaN,NaN,,4.0,No,Yes,1.0,0.485676,0.642901
4,Feature,5,-2.595294,51.442812,5,B12,Bedminster Parade,5,358723,171704,...,NaN,NaN,NaN,,1.0,Yes,No,96.0,0.485582,0.642897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Feature,696,-2.527912,51.465290,696,None,Rodney Road junction with Whiteway Road,707,363424,174168,...,1.722384e+12,NaN,NaN,None,NaN,None,None,NaN,0.485956,0.642959
696,Feature,697,-2.535671,51.461542,697,None,Whiteway Road junction with A420,708,362882,173755,...,1.722470e+12,NaN,NaN,None,NaN,None,None,NaN,0.485913,0.642949
697,Feature,698,-2.530569,51.458571,698,None,Air Balloon Hill Primary School,709,363234,173422,...,1.722384e+12,NaN,NaN,None,NaN,None,None,NaN,0.485941,0.642940
698,Feature,699,-2.525999,51.454194,699,None,Nags Head Hill,710,363548,172933,...,1.722384e+12,NaN,NaN,None,NaN,None,None,NaN,0.485967,0.642928


<h3>3. Data Processing</h3>

In [39]:
# create new dataframe from selected columns
selected_columns = ['id','location','LocationClass','Latitude (Normalised)','Longitude (Normalised)','pollutants']
new_df = features_df[selected_columns]

# rename columns
new_df = new_df.rename(columns={'id':'ID','location':'Location','LocationClass':'Location Type','pollutants':'Pollutants'})

new_df

,ID,Location,Location Type,Latitude (Normalised),Longitude (Normalised),Pollutants
0,1,Withywood School,Urban Traffic,0.485401,0.642799,BTX NO2
1,2,Colston Avenue,Urban Traffic,0.485573,0.642929,NO2
2,3,Blackboy Hill,Urban Traffic,0.485478,0.642970,NO2
3,4,Three Lamps,Urban Traffic,0.485676,0.642901,NO2
4,5,Bedminster Parade,Urban Traffic,0.485582,0.642897,NO2
...,...,...,...,...,...,...
695,696,Rodney Road junction with Whiteway Road,None,0.485956,0.642959,NO2
696,697,Whiteway Road junction with A420,None,0.485913,0.642949,NO2
697,698,Air Balloon Hill Primary School,None,0.485941,0.642940,NO2
698,699,Nags Head Hill,None,0.485967,0.642928,NO2


In [40]:
new_df.dtypes

ID                          int64
Location                   object
Location Type              object
Latitude (Normalised)     float64
Longitude (Normalised)    float64
Pollutants                 object
dtype: object

In [54]:
# set datatype of columns to string for columns set as object 
new_df['Location'] = new_df['Location'].astype('string')
new_df['Location Type'] = new_df['Location Type'].astype('string')
new_df['Pollutants'] = new_df['Pollutants'].astype('string')

In [55]:
new_df.dtypes

ID                                 int64
Location                  string[python]
Location Type             string[python]
Latitude (Normalised)            float64
Longitude (Normalised)           float64
Pollutants                string[python]
dtype: object

In [58]:
new_df.head(50)

,ID,Location,Location Type,Latitude (Normalised),Longitude (Normalised),Pollutants
0,1,Withywood School,Urban Traffic,0.485401,0.642799,BTX NO2
1,2,Colston Avenue,Urban Traffic,0.485573,0.642929,NO2
2,3,Blackboy Hill,Urban Traffic,0.485478,0.642970,NO2
3,4,Three Lamps,Urban Traffic,0.485676,0.642901,NO2
4,5,Bedminster Parade,Urban Traffic,0.485582,0.642897,NO2
5,6,Church Road,Urban Traffic,0.485784,0.642940,NO2
6,7,St. Andrew's Rd,Urban Traffic,0.485016,0.643059,NO2
7,8,Higham Street,Urban Background,0.485671,0.642902,NO2
8,9,B.R.I.,Urban Traffic,0.485581,0.642942,NO2
9,10,Bath Road,Urban Traffic,0.485781,0.642890,NO2


<h2>Part 2 - Load</h2>
<h3>4. Loading to a CSV file</h3>

In [ ]:
new_df.to_csv('air_quality_data.csv', index=False) # index removes the indices